In [1]:
import os, sys

print("Python:", sys.executable)
print("PWD:", os.getcwd())

try:
    import finn
    print("FINN imported from:", finn.__file__)
except Exception as e:
    print("FINN import FAILED:", e)


Python: /usr/bin/python3
PWD: /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/notebooks/ellipse_regression_project
FINN imported from: None


In [2]:
import sys
print("\n".join(sys.path))


/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/tmp/home_dir/.local/lib/python3.10/site-packages
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/deps/qonnx/src
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/deps/finn-experimental/src
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/deps/brevitas/src
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/deps/pyverilator
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/src
/usr/local/lib/python3.10/dist-packages
/workspace/src/dataset-loading
/usr/lib/python3/dist-packages


In [3]:
import os
assert os.path.isdir("/tools/Xilinx/Vivado"), "Vivado not mounted"
print("Vivado path OK")
print("Vivado versions:", os.listdir("/tools/Xilinx/Vivado"))

Vivado path OK
Vivado versions: ['2023.2']


In [4]:
!echo "HLS_PATH = $HLS_PATH"
!which vitis_hls
!vitis_hls -version


HLS_PATH = /tools/Xilinx/Vitis_HLS/2023.2
/tools/Xilinx/Vitis_HLS/2023.2/bin/vitis_hls
Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2023.2 (64-bit)
SW Build 4023990 on Oct 11 2023
IP Build 4028589 on Sat Oct 14 00:45:43 MDT 2023
Tool Version Limit: 2023.10
Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
Copyright 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.
SharedData Build 4025554 on Tue Oct 10 17:18:54 MDT 2023



In [5]:
from finn.builder.build_dataflow import build_dataflow_cfg
from finn.builder.build_dataflow_config import DataflowBuildConfig
from finn.util.basic import make_build_dir
from qonnx.core.modelwrapper import ModelWrapper  # <--- QONNX wrapper

print("✓ FINN imports OK")

✓ FINN imports OK


In [6]:
model_path = "ellipse_regression_hw_ready.onnx"
build_dir = make_build_dir("build_kv260")

print("Model:", model_path)
print("Build dir:", build_dir)


Model: ellipse_regression_hw_ready.onnx
Build dir: /tmp/finn_dev_hritik/build_kv260z1s83hf2


In [7]:
model = ModelWrapper(model_path)
model.check_all_tensor_shapes_specified()
print("Model loaded successfully with QONNX wrapper")

Model loaded successfully with QONNX wrapper


In [8]:
import os
os.environ["FINN_NUM_HLS_JOBS"] = "1"


In [9]:
#cfg = DataflowBuildConfig(
    #output_dir=build_dir,
    #mvau_wwidth_max=80,
    #target_fps=2000,
    #synth_clk_period_ns=5.0,
    #fpga_part="xck26-sfvc784-2LV-c",
    #board="kv260",
    #shell_flow_type="vitis",
    #generate_outputs=[
     #   "estimate_reports",
     #   "bitfile",
     #   "driver"
    #],
#)

In [9]:
from finn.builder.build_dataflow_config import (
    DataflowBuildConfig,
    DataflowOutputType,
    ShellFlowType
)

cfg = DataflowBuildConfig(
    output_dir=build_dir,

    # Safe MVAU limits
    mvau_wwidth_max=80,

    # REALISTIC targets (important)
    target_fps=100,                 # start low
    synth_clk_period_ns=10.0,        # 100 MHz

    # FINN-supported board
    board="ZCU104",                  # <-- KV260 equivalent

    # REQUIRED shell flow
    shell_flow_type=ShellFlowType.VIVADO_ZYNQ,

    generate_outputs=[
        DataflowOutputType.ESTIMATE_REPORTS,
        DataflowOutputType.BITFILE,
        DataflowOutputType.PYNQ_DRIVER,
    ],
)


In [10]:
build_dataflow_cfg(model_path, cfg)


Building dataflow accelerator from ellipse_regression_hw_ready.onnx
Intermediate outputs will be generated in /tmp/finn_dev_hritik
Final outputs will be generated in /tmp/finn_dev_hritik/build_kv260z1s83hf2
Build log is at /tmp/finn_dev_hritik/build_kv260z1s83hf2/build_dataflow.log
Running step: step_qonnx_to_finn [1/19]
Running step: step_tidy_up [2/19]
Running step: step_streamline [3/19]
Running step: step_convert_to_hw [4/19]
Running step: step_create_dataflow_partition [5/19]
Running step: step_specialize_layers [6/19]
Running step: step_target_fps_parallelization [7/19]
Running step: step_apply_folding_config [8/19]
Running step: step_minimize_bit_width [9/19]
Running step: step_generate_estimate_reports [10/19]
Running step: step_hw_codegen [11/19]
Running step: step_hw_ipgen [12/19]
Running step: step_set_fifo_depths [13/19]
Running step: step_create_stitched_ip [14/19]
Running step: step_measure_rtlsim_performance [15/19]
Running step: step_out_of_context_synthesis [16/19]
Run

0

In [11]:
print("Build complete. Output files:")
for root, dirs, files in os.walk(build_dir):
    for f in files:
        if f.endswith((".bit", ".hwh")):
            print(os.path.join(root, f))

Build complete. Output files:
/tmp/finn_dev_hritik/build_kv260z1s83hf2/bitfile/finn-accel.hwh
/tmp/finn_dev_hritik/build_kv260z1s83hf2/bitfile/finn-accel.bit
